In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [158]:
from ast import literal_eval
from math import log10, log

from IPython.display import Markdown, display, HTML
import pandas as pd

past_queries = ['criativo','Instagram','reprodução','real','hostil']

In [178]:
inv_index = pd.read_csv("../output/inverted_index.csv").set_index(['word']).drop('vivoÉ') #remove tokenization mistake
inv_index["doc_id:freq"] = inv_index["doc_id:freq"].apply(lambda x: literal_eval(x))
inv_index.head(20)

,doc_id:freq
word,
juíza,"[(0, 2), (1, 1)]"
federal,"[(0, 1), (1, 1), (2, 1), (6, 2), (14, 1), (36,..."
Ivani,"[(0, 1), (1, 1)]"
Silva,"[(0, 3), (1, 1), (5, 1), (13, 2), (25, 1), (72..."
Luz,"[(0, 3), (1, 1), (124, 1)]"
Brasília,"[(0, 1), (7, 1), (32, 1), (34, 1), (43, 1), (4..."
proibiu,"[(0, 1), (1, 1), (118, 1), (161, 1)]"
caráter,"[(0, 1), (14, 1), (35, 1), (59, 1), (88, 1), (..."
liminar,"[(0, 1), (1, 3), (118, 1), (216, 1)]"


In [154]:
def mim(**kwargs):
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]
    
    return n_ab/(n_a * n_b)

def dice_coef(**kwargs):
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]

    return n_ab/(n_a + n_b)

def emim(**kwargs):
    N = 10e6 if 'N' not in kwargs else kwargs["N"]
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]
    result = 0 if n_ab == 0 else \
             n_ab * log10(N * (n_ab / (n_a * n_b)))
    
    return result

def chi_square(**kwargs):
    N = 10e6 if 'N' not in kwargs else kwargs["N"]
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]

    return pow((n_ab - (1/N) * n_a * n_b),2) / (n_a * n_b)

In [98]:
del get_n_most_related

In [179]:
from bisect import insort_left


def n_most_related(n=10,**kwargs):
    metric = kwargs.pop("metric")
    index = kwargs.pop("df")
    A = kwargs.pop("A")    
    words = index.drop(A).reset_index()["word"].values
    rank = []
    
    for word in words:
        n_a = index.xs(A).values[0]
        n_b = index.xs(word).values[0]
        n_ab = [val1 for (val1,val2) in n_a if val1 \
            in (val1 for (val1,val2) in n_b)]
        kwargs["n_ab"] = len(n_ab)
        kwargs["n_a"] = len(n_a)
        kwargs["n_b"] = len(n_b)   
        insort_left(rank, (metric(**kwargs),word))
        rank = rank[-n:]
    
    return rank

In [180]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='Instagram',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'Instagram'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns=columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'Instagram'

MIM,EMIM,Chi-Squared,Dice
êxito,sociais,publicadas,publicadas
ânsias,fazer,contínuos,contínuos
votarão,onde,Gallup,Gallup
votará,dois,tarifas,tarifas
vinhos,é,exibe,exibe
viatura,longe,êxito,jogam
velinhas,redes,ânsias,império
utópico,nada,votarão,gás
usos,deu,votará,aço
urbana/rural,5,vinhos,encerrou


***

In [181]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='criativo',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'criativo'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'criativo'

MIM,EMIM,Chi-Squared,Dice
ângulos,cheque,ângulos,ângulos
virarem,conhecidas,virarem,virarem
violou,acredito,violou,violou
viciado,mental,viciado,viciado
vestiam,trabalhadores,vestiam,vestiam
veste,parlamentares,veste,veste
vermelho-sangue,efeitos,vermelho-sangue,vermelho-sangue
vaidades,cara,vaidades,vaidades
vaginas,podemos,vaginas,vaginas
usarem,acontecer,usarem,usarem


***

In [182]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='reprodução',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'reprodução'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'reprodução'

MIM,EMIM,Chi-Squared,Dice
útero,forma,médicas,médicas
úteis,pessoas,costumes,costumes
ônibusFoi,sobre,útero,vazio
ária,é,úteis,valem
workshops,faz,ônibusFoi,usuário
votam,momento,ária,preparação
vivências,cada,workshops,bruto
visualizações,vida,votam,oferecem
virgindade,diz,vivências,influentes
verbalmente,onde,visualizações,experiências


***

In [183]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='real',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'real'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'real'

MIM,EMIM,Chi-Squared,Dice
úteis,é,justamente,significa
ítalo-americano,sobre,sobre,justamente
ítalo-americana,anos,significa,mostrar
índole,ser,salto,ficção
êxito,tempo,estranhas,verdade
éticas,fazer,cristão,realidade
ânus,É,animação,perder
ânsias,ainda,Cohen,dizem
ávido,disso,ficção,chama
áspero,assim,verdade,acreditam


***

In [185]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='hostil',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'hostil'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'hostil'

MIM,EMIM,Chi-Squared,Dice
utensílios,fere,fere,fere
tornasse,delicado,utensílios,utensílios
tensionamento,judiciais,tornasse,tornasse
suspendendo,Escola,tensionamento,tensionamento
surfar,Chegou,suspendendo,suspendendo
supermovimentada,defendeu,surfar,surfar
situação-limite,causas,supermovimentada,supermovimentada
servidora,posse,situação-limite,situação-limite
rosas,Militar,servidora,servidora
ribeirinhas,Todos,rosas,rosas


***